In [1]:
import numpy as np
import os
import cv2
import mediapipe as mp
import subprocess
mp_selfie_segmentation = mp.solutions.selfie_segmentation
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
print(cv2.getBuildInformation())


General configuration for OpenCV 4.8.0 =====================================
  Version control:               4.8.0

  Platform:
    Timestamp:                   2023-06-30T11:22:01Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.24.2
    CMake generator:             Visual Studio 14 2015
    CMake build tool:            MSBuild.exe
    MSVC:                        1900
    Configuration:               Debug Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (7 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX
      AVX2 (35 files):           + SSSE3 SSE4_1 

In [3]:
video_directory = "E:\\course_work\\Dissertation\\back_001"

all_files_in_directory = os.listdir(video_directory)

video_files = [file for file in all_files_in_directory if file.endswith(('.mkv'))]

videos = []

for video_file in video_files:
    video_file_path = os.path.join(video_directory, video_file)
    output_file_path = os.path.splitext(video_file_path)[0] + '.mp4'
    subprocess.run(['ffmpeg', '-i', video_file_path, output_file_path])
    videos.append(output_file_path)


In [4]:
  
output_folder = "E:\\course_work\\Dissertation\\pose_estim" 
output_extension = ".mp4"

# Set the video writer parameters
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Specify the video codec (e.g., "mp4v" for MP4)

frame_size = (1280, 720)


# Estimation Keypoint

<img src="https://www.mdpi.com/applsci/applsci-13-02700/article_deploy/html/images/applsci-13-02700-g001.png" style="height=200px" >

# Jumping

In [5]:
def calculate_vertical_displacement(a, b):
    """Calculate the vertical displacement between points a and b"""
    return b - a

# Squating

In [6]:
def calculate_angle(a, b, c):
    """Calculate the angle between points a, b, and c"""
    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    #cosine_angle = np.clip(cosine_angle, -1, 1)
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)

In [16]:
import time
with mp_pose.Pose(min_detection_confidence=0.8 , min_tracking_confidence=0.8) as pose,mp_selfie_segmentation.SelfieSegmentation(model_selection=0) as selfie_segmentation:
    for vid in videos:
        video = cv2.VideoCapture(vid)
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        squat_detected = False
        jump_detected = False
        baseline = None
        jump_threshold = 0.015
        filename = os.path.splitext(os.path.basename(vid))[0]
        output_path = os.path.join(output_folder, filename + output_extension)
        original_fps = video.get(cv2.CAP_PROP_FPS)
        original_frame_size = (
            int(video.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        )
        video_writer = cv2.VideoWriter(output_path, fourcc, original_fps, frame_size)
        while True:
            ret,frame = video.read()
            if not ret:
                break

            resized_frame = cv2.resize(frame, frame_size)
            image = cv2.cvtColor(resized_frame,cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            segmentation_results = selfie_segmentation.process(image)
            image.flags.writeable = True
            
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                # Squating
                # right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y])
                # left_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y])
                # right_knee = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y])
                # left_knee = np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y])
                # right_shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y])
                # left_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y])
                # right_elbow = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y])
                # left_elbow = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y])
                # right_heel = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HEEL].x, landmarks[mp_pose.PoseLandmark.RIGHT_HEEL].y])
                # left_heel = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HEEL].x, landmarks[mp_pose.PoseLandmark.LEFT_HEEL].y])
                
                
                # if hip_knee_angle < 90 and shoulder_hip_angle < 130 and shoulder_angle < 70 and shoulder_angle > 60 and elbow_angle > 90 and elbow_angle < 100 :# Adjust the threshold values as needed
                #     if right_heel[1] >= right_knee[1] and left_heel[1] >= left_knee[1]:
                #         squat_detected = True

                right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y])
                right_knee = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y])
                left_knee = np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y])
                right_shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y])
                left_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y])
                right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y])
                right_elbow = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y])
                right_wrist = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y])
                
                # Calculate angles
                elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                hip_knee_angle = calculate_angle(right_hip, right_knee, left_knee)
                shoulder_hip_angle = calculate_angle(right_shoulder, right_hip, left_hip)
                shoulder_angle = calculate_angle(right_shoulder, left_shoulder, right_hip)
                
                # Detect squats
                squat_condition1 = hip_knee_angle < 110
                squat_condition2 = shoulder_hip_angle < 130
                squat_condition3 = shoulder_angle < 70
                squat_condition4 = elbow_angle > 90
                squat_condition5 = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL].y >= landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y
                squat_condition6 = landmarks[mp_pose.PoseLandmark.LEFT_HEEL].y >= landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y
                if squat_condition1 and squat_condition2 and squat_condition3 and squat_condition4 and squat_condition5 and squat_condition6:  # Ensuring heels are grounded
                    squat_detected = True
                
                # Detect jumps
                # jump_condition1 = hip_knee_angle < 150
                # jump_condition2 = shoulder_hip_angle < 180
                # jump_condition3 = shoulder_angle < 90
                # jump_condition4 = elbow_angle > 80
                # jump_condition5 = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL].y < landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y
                # jump_condition6 = landmarks[mp_pose.PoseLandmark.LEFT_HEEL].y < landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y
                # if jump_condition1 and jump_condition2 and jump_condition3 and jump_condition4 and jump_condition5 and jump_condition6 and not squat_detected:  
                #     # Only detect a jump if a squat hasn't been detected in the same frame
                #     jump_detected = True


                # Jumping

                hip_y = (landmarks[27].y + landmarks[28].y) / 2
    
                if baseline is None:
                    baseline = hip_y
                else:
                    # Calculate the distance of the hips from the baseline
                    hip_distance = calculate_vertical_displacement(baseline,hip_y)

                    if hip_distance > jump_threshold :
                        jump_detected = True
                    baseline = hip_y

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                #Frame Normalization
                landmarks_array = np.array([(lmk.x, lmk.y, lmk.z) for lmk in landmarks])
                # Normalize the landmarks
                mean = np.mean(landmarks_array, axis=0)
                std = np.std(landmarks_array, axis=0)
                normalized_landmarks = (landmarks_array - mean) / std

            condition = np.stack((segmentation_results.segmentation_mask,) * 3, axis=-1) > 0.1
            # this two will blurred background 
            blurred_image = cv2.GaussianBlur(image, (99, 99), 30)
            output_image = np.where(condition, image, blurred_image)

            # this below lines change background to green color
            # background_color = np.zeros_like(image)
            # background_color[:] = [0, 255, 0]  # Green in BGR
            # output_image = np.where(condition, image, background_color)
            

            video_writer.write(output_image)
        if squat_detected and not jump_detected:
            print(f"Squat detected in video: {vid}")
        if jump_detected and squat_detected:
            print(f"Jump detected in video: {vid}")
        # else:
        #     print(f"nothing detected in video: {vid}")
        video.release()
        video_writer.release()



Squat detected in video: E:\course_work\Dissertation\back_001\back_001_m01_20230224094505.mp4
Squat detected in video: E:\course_work\Dissertation\back_001\back_001_m01_20230224094600.mp4
Squat detected in video: E:\course_work\Dissertation\back_001\back_001_m01_20230224094626.mp4
Squat detected in video: E:\course_work\Dissertation\back_001\back_001_m01_20230224094649.mp4
Squat detected in video: E:\course_work\Dissertation\back_001\back_001_m01_20230224094705.mp4
Squat detected in video: E:\course_work\Dissertation\back_001\back_001_m01_20230224094722.mp4
Squat detected in video: E:\course_work\Dissertation\back_001\back_001_m01_20230224094739.mp4
Squat detected in video: E:\course_work\Dissertation\back_001\back_001_m01_20230224094754.mp4
Squat detected in video: E:\course_work\Dissertation\back_001\back_001_m01_20230224094811.mp4
Squat detected in video: E:\course_work\Dissertation\back_001\back_001_m01_20230224094826.mp4
Squat detected in video: E:\course_work\Dissertation\back_00